## Classification problem

This is an example of a classification problem.
The results are used for the notebooks on measuring the performance of classification models (for the book on DL).

**Import libraries** and **set the seed**

In [ ]:
import numpy as np
import pandas as pd

nint = 113
np.random.seed(nint)

#### Load the data

Breast cancer data:

- binary classification
- 30 features, numeric

In [ ]:
from sklearn.datasets import load_breast_cancer

(X, y) = load_breast_cancer(return_X_y=True)

In [ ]:
print("N. of examples", len(y))

In [ ]:
unique, counts = np.unique(y, return_counts=True)
print(np.asarray((unique, counts)).T)

#### Subset columns

Take a random subset of features:

In [ ]:
import random

n_vars = 8 ## n. of features to select (randomly)
lst = list(range(0,X.shape[1]))

In [ ]:
random.seed(nint)
selected_cols = random.sample(lst, n_vars)

In [ ]:
X = X[:,selected_cols]

#### Data normalization

Center and scale the matrix of features `X`

In [ ]:
random.seed(nint)
print(random.random())

In [ ]:
avg = np.mean(X, axis=0)
std = np.std(X, axis=0)

In [ ]:
X_norm = (X-avg)/std

In [ ]:
print("Mean of 1st feature:",X_norm[:,0].mean())
print("Standard deviation of 1st feature:", X_norm[:,0].std())

#### Training / test data split

Randomly split the data in training (80%) and test set (20%)

In [ ]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, random_state=nint)

In [ ]:
print("N. of training examples:", len(X_train))

In [ ]:
print("N. of test examples:", len(X_test))

In [ ]:
y_train[0:10]

### Logistic regression model

Fir the logistic regression model:

In [ ]:
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression(random_state=nint)

In [ ]:
# fit the model with data
logreg.fit(X_train, y_train)

#### Get predictions on the test set

In [ ]:
y_pred = logreg.predict(X_test)

#### Measure model performance

In [ ]:
# import the metrics class
from sklearn import metrics

cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

In [ ]:
lr_probs = logreg.predict_proba(X_test)

In [ ]:
#this cell mounts the user's google drive in the specified folder,
#but only once (doing more than once would generate an error)
import os

gdrive_folder = '/content/gdrive'
project_folder = '/content/gdrive/MyDrive/projects/book_DL' ## !! IMPORTANT: change this depending on data iteration !!

if not os.path.isdir(gdrive_folder):
  from google.colab import drive
  drive.mount(gdrive_folder)

In [ ]:
y_all = np.vstack((y_test, y_pred)).T

In [ ]:
df = pd.DataFrame(np.hstack((y_all, lr_probs)), columns=['y_test', 'y_pred', 'prob_0', 'prob1'])
df.head()

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
def writeout_results(res, filename):

    basedir = os.path.dirname(filename)
    if os.path.isdir(basedir):
          res.to_csv(filename, mode='w', header=True, index=False)
          return "Creating file '{}' and writing results to it".format(os.path.basename(filename))
    else:
          os.makedirs(os.path.dirname(filename))
          res.to_csv(filename, mode='w', header=True, index=False)
          return "Creating folder '{}' and writing results to file {}".format(basedir, os.path.basename(filename))

In [ ]:
## the model object is used to extract predictions
## if not reinstantiated, I believe that the model object keeps being over
## epochs in the above for loop; therefore the model object after the loop
## containes the fully trained model to be used for predictions

print(" - saving predictions")
fname = os.path.join(project_folder, "predictions.csv")
print("writing results to: ", fname)
writeout_results(df, fname)